In [1]:
from typing import Any, Dict, Optional, Tuple
import torch
from datasets import Dataset as HfDataset
from modelscope import (AutoConfig, AutoModelForCausalLM, AutoTokenizer,
                        MsDataset)
from torch import dtype as Dtype
from transformers.utils.versions import require_version

from swift.llm import (LoRATM, Template, TemplateType, dataset_map,
                       get_dataset, get_model_tokenizer, get_template,
                       print_example, register_dataset, register_model,
                       register_template)
from swift.utils import get_logger
logger = get_logger()


2024-01-14 15:36:16,653 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-01-14 15:36:16,656 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-01-14 15:36:16,683 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 dc3cda2b56a86b0242cd145399a14071 and a total number of 953 components indexed


In [2]:
def get_dpopenHermes_model_tokenizer(model_dir: str,
                                 torch_dtype: Dtype,
                                 model_kwargs: Dict[str, Any],
                                 load_model: bool = True,
                                 **kwargs):
    model_config = AutoConfig.from_pretrained(model_dir)
    model_config.torch_dtype = torch_dtype
    logger.info(f'model_config: {model_config}')
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = None
    if load_model:
        model = AutoModelForCausalLM.from_pretrained(
            model_dir,
            config=model_config,
            torch_dtype=torch_dtype,
            **model_kwargs)
    return model, tokenizer

In [3]:
model, tokenizer = get_dpopenHermes_model_tokenizer(
    '/home/css/models/DPOpenHermes-7B-v2', 
    torch.bfloat16,
    {},
    load_model=False
)



[INFO:swift] model_config: MistralConfig {
  "_name_or_path": "/home/css/models/DPOpenHermes-7B-v2",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.36.2",
  "use_cache": false,
  "vocab_size": 32002
}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import jsonlines
from tqdm.notebook import tqdm

dict_list = []
version = "1"
with jsonlines.open(f"data{version}.jsonl", mode="r") as file_jsonl:
    for item in file_jsonl.iter():
        dict_list.append(item)

lens1 = []
lens2 = []

for item in tqdm(dict_list):
    lens1.append(len(tokenizer(item["query"])["input_ids"]))
    # lens2.append(len(tokenizer.tokenize(item["query"])))



  0%|          | 0/12192 [00:00<?, ?it/s]

In [5]:
# 3705

sorted(lens1, reverse=True)[:10]


[3593, 3568, 3561, 3555, 3554, 3540, 3536, 3536, 3535, 3530]

修改较长的句子

显卡 24GB

7B模型，长度控制在3600以下 


In [23]:
del_ids = []
for i, item in enumerate(lens1):
    if item == 3604:
        del_ids.append(i)

del_ids


[783, 9321]

In [26]:
dict_list[9321]["query"]

'According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.\n\nCLAIM:\nPublication date: 2020-03-28\nContent: French State approved a medical protocol with Plaquenil, lopinavir/ritonavir to cure COVID-19, which is more expensive than Pr. Raoult’s protocol with chloroquine.\n\nPRIOR KNOWLEDGE:\nInformation 1:\nPublication date: 2020-05-22\nTitle: He Was a Science Star. Then He Promoted a Questionable Cure for ...\nContent:\nRaoult reviewed that evidence in a 2007 paper, concluding that chloroquine and hydroxychloroquine might be “an interesting weapon to face present and future infectious diseases worldwide.” This winter, as the spread of SARS-CoV-2 began to take on the contours of a pandemic, he surveyed the data that had begun coming out of China. An early report on chloroquine showed good results in

In [27]:
print(dict_list[9321]["query"])


According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.

CLAIM:
Publication date: 2020-03-28
Content: French State approved a medical protocol with Plaquenil, lopinavir/ritonavir to cure COVID-19, which is more expensive than Pr. Raoult’s protocol with chloroquine.

PRIOR KNOWLEDGE:
Information 1:
Publication date: 2020-05-22
Title: He Was a Science Star. Then He Promoted a Questionable Cure for ...
Content:
Raoult reviewed that evidence in a 2007 paper, concluding that chloroquine and hydroxychloroquine might be “an interesting weapon to face present and future infectious diseases worldwide.” This winter, as the spread of SARS-CoV-2 began to take on the contours of a pandemic, he surveyed the data that had begun coming out of China. An early report on chloroquine showed good results in vitro. In m